In [1]:
from dotenv import load_dotenv
import os

In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import  GoogleGenerativeAIEmbeddings

c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load environment variables from .env file
load_dotenv()

True

In [4]:
# Access the Gemini API Key from environment variable
api_key = os.getenv("GOOGLE_API_KEY")

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.7, google_api_key=api_key)
embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-005", google_api_key=api_key)

In [6]:
from langchain_community.retrievers import WikipediaRetriever

In [7]:
# Wikipedia Retriever
retriever = WikipediaRetriever(top_k_results=2, lang="en")
query = "the geopolitical history of india and pakistan from the perspective of a chinese"
docs = retriever.invoke(query)

In [8]:
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")


--- Result 1 ---
Content:
The United States had been providing military aid and economic assistance to Pakistan for various purposes since 1948. Recently U.S. stopped military aid to Pakistan, which was about US$2 billion per year. With U.S. military assistance suspended in 2018 and civilian aid reduced to about $300 million for 2022, Pakistani authorities have turned to other countries for help.


== History ==
From 1947 to 1958, under civilian leadership, the United States provided Pakistan with modest economic aid and limited military assistance. During this period, Pakistan became a member of the South East Asian Treaty Organization (SEATO) and the Central Treaty Organization (CENTO), after a Mutual Defence Assistance Agreement signed in May 1954, which facilitated increased levels of both economic and military aid from the U.S.
In 1958, Ayub Khan led Pakistan's first military coup, becoming Chief Martial Law Administrator (CMLA) and later President until 1969. During his tenure, 

In [9]:
# Vector Store Retriever with Chroma + Gemini Embeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

In [10]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [15]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\emon1\AppData\Local\Temp\ipykernel_3016\3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [17]:
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="my_collection"
)

In [18]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [19]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


In [20]:
# FAISS with HuggingFace Embeddings
from langchain_community.vectorstores import FAISS

In [21]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [22]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

In [23]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "lambda_mult": 0.5}
)

In [24]:
query = "What is langchain?"
results = retriever.invoke(query)

In [25]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
Embeddings are vector representations of text.


In [26]:
# MultiQueryRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever

In [27]:
all_docs = [
    Document(page_content="Regular walking boosts heart health...", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps...", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for repair...", metadata={"source": "H3"}),
    Document(page_content="Mindfulness lowers cortisol...", metadata={"source": "H4"}),
    Document(page_content="Drinking water boosts metabolism...", metadata={"source": "H5"}),
]

In [30]:
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embeddings)
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [37]:
query = "How to improve energy levels and maintain balance?"

In [40]:
similarity_results = similarity_retriever.invoke(query)

In [41]:
print("\nSimilarity Results:")
for i, doc in enumerate(similarity_results):
    print(f"\n--- {i+1} ---\n{doc.page_content}")


Similarity Results:

--- 1 ---
Consuming leafy greens and fruits helps...

--- 2 ---
Regular walking boosts heart health...

--- 3 ---
Drinking water boosts metabolism...

--- 4 ---
Mindfulness lowers cortisol...

--- 5 ---
Deep sleep is crucial for repair...
